Aqui montaremos a rede para avaliar os dados para a tarefa de classificação multiclasse com a utilização dos dados brutos

Importar as bibliotecas e os dados do conjunto MNIST

In [11]:
import json

from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

# Carregar os dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

Normalizar e redimensionar os dados

In [12]:
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

Converter os rótulos para one-hot encoding

`to_categorical(rótulos, número de classes)`

In [13]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

Construir o modelo

In [14]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

c:\Users\mtoma\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


4º Compilaremos o modelo

In [15]:
# Compilar o modelo
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

- Criando e salvando os hiperparâmetros da arquitetura e da inicialização em formato json em arquivos-brutos/hiperparametros.json

- Salvando os pesos iniciais

In [16]:
# HIPERPARÂMETROS
hiperparametros = {
    "arquitetura": {
        "camadas": [
            {"tipo": "Conv2D", "filtros": 32, "kernel_size": 3, "ativacao": "relu"},
            {"tipo": "MaxPooling2D", "pool_size": 2},
            {"tipo": "Conv2D", "filtros": 64, "kernel_size": 3, "ativacao": "relu"},
            {"tipo": "MaxPooling2D", "pool_size": 2},
            {"tipo": "Flatten"},
            {"tipo": "Dense", "unidades": 64, "ativacao": "relu"},
            {"tipo": "Dense", "unidades": 10, "ativacao": "softmax"}
        ]
    },
    "inicializacao": {
        "optimizer": "adam",
        "loss": "categorical_crossentropy",
        "metrics": ["accuracy"]
    }
}

# Serializando os hiperparâmetros em uma string JSON
hiperparametros_json = json.dumps(hiperparametros, indent=4)

# Escrevendo a string JSON em um arquivo
with open("arquivos-brutos/hiperparametros.json", "w") as arquivo:
    arquivo.write(hiperparametros_json)


# PESOS INICIAIS
model.save_weights('arquivos-brutos/pesos_iniciais.weights.h5')

Treinar o modelo

In [17]:
historico = model.fit(x_train, y_train, epochs=15, batch_size=64)

Epoch 1/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 23s 21ms/step - accuracy: 0.8225 - loss: 0.5412
Epoch 2/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.9729 - loss: 0.0880
Epoch 3/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 18ms/step - accuracy: 0.9802 - loss: 0.0617
Epoch 4/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 18s 19ms/step - accuracy: 0.9852 - loss: 0.0491
Epoch 5/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9858 - loss: 0.0438
Epoch 6/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9885 - loss: 0.0372
Epoch 7/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9898 - loss: 0.0328
Epoch 8/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9909 - loss: 0.0294
Epoch 9/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9919 - loss: 0.0257
Epoch 10/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9917 - loss: 0.0259
Epoch 11/15
938/938 ━━━━━━━━━━━━━━━━━━━━ 17s 18ms/step - accuracy: 0.9923 - loss: 0.0237
Epoch 12/15
938/938 ━━━━━━━━━━

- Salvando os pesos finais da rede 
- Salvando o histórico de perda para cada iteração
- Salvando as saídas produzidas pela rede para cada um dos dados de teste

In [20]:
# PESOS FINAIS
model.save_weights('arquivos-brutos/pesos_finais.weights.h5')

# ERRO DE CADA ITERAÇÃO
perdas = historico.history['loss']

# Salvando o histórico de perda em um arquivo JSON
with open('arquivos-brutos/historico_perda.json', 'w') as f:
    json.dump(perdas, f)


# SAÍDAS PRODUZIDAS
# Fazendo inferência com o modelo treinado para obter as saídas
saidas = model.predict(x_test)

# Convertendo as saídas para uma lista para serialização
saidas_lista = saidas.tolist()

# Salvando as saídas em um arquivo JSON
with open('arquivos-brutos/saidas_teste.json', 'w') as f:
    json.dump(saidas_lista, f)

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


6º Testaremos o modelo

In [21]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9916 - loss: 0.0269
Test accuracy: 0.9937999844551086


NOTAS:

Para criar um modelo de CNN (Rede Neural Convolucional) para o conjunto de dados MNIST, que é um conjunto de dígitos manuscritos de 0 a 9, e portanto um problema de classificação multiclasse, seguiremos os seguintes passos em pseudocódigo:

Importar as bibliotecas necessárias.
Carregar o conjunto de dados MNIST.
Pré-processar os dados:
Normalizar os valores dos pixels para o intervalo de 0 a 1.
Redimensionar as imagens, se necessário, para garantir que todas tenham o mesmo tamanho.
Converter os rótulos (labels) para uma representação "one-hot" para a classificação multiclasse.
Construir o modelo da CNN:
Adicionar camadas convolucionais para extrair características das imagens.
Utilizar camadas de pooling para reduzir a dimensionalidade.
Adicionar camadas de dropout para reduzir o overfitting.
Incluir uma camada achatada (Flatten) para transformar os mapas de características em um vetor.
Adicionar uma ou mais camadas densas (fully connected) para a classificação.
A última camada densa deve ter um número de neurônios igual ao número de classes (10 para MNIST) e utilizar a função de ativação softmax para a classificação multiclasse.
Compilar o modelo:
Escolher um otimizador (por exemplo, Adam).
Definir a função de perda (loss function) como 'categorical_crossentropy' para classificação multiclasse.
Especificar métricas para monitorar, como 'accuracy'.
Treinar o modelo com os dados de treinamento.
Avaliar o modelo com os dados de teste para verificar a precisão.
Agora, vamos transformar esse pseudocódigo em código Python usando TensorFlow e Keras: